In [2]:
import os
import sys
import pandas as pd
import concurrent.futures

sys.path.append('../')

from project.utils import download_images, save_image_locally, get_image_list

## Notebook Purpose
<b> This is notebook number 2 </b>

This notebook looks into the recently downloaded csv and downloads images to the specified directory for labeling

### Notebook Order
1. getData
2. downloadData

In [10]:
##Put this where you want your data to go
dataDir = "./data/"##Where you want to save your data/and where the csv is
os.listdir(dataDir)

['image_prompt_tag_data.csv', 'results', 'testimgs']

In [12]:
df = pd.read_csv(os.path.join(dataDir, "image_prompt_tag_data.csv")).rename(columns={"original_level": "label"})
df.head()

,download_url,url,id,label,tags,prompt
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,3941c5f2-d694-44bf-a6e3-b2158f7fd038,7482165,XXX,"woman, brown hair, chair, closed eyes, coverin...",iphone photograph of sleepy tired pale freckle...
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,15f5d724-8831-4b5f-8e07-491696366155,6429527,PG,"woman, bow, bracelet, breasts, jewelry, long h...","1girl, solo, portrait, simple background, dark..."
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,3d64c9d8-d812-4e93-97e1-4a318d01afa4,7483116,X,"woman, breasts, brown eyes, brown hair, long h...","((nude, topless, nipples, white panties, white..."
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,de196b97-127f-4616-8133-52bd3b8f8494,7481494,PG13,"woman, black hair, cowboy shot, curly hair, fe...","(sfw:1.2) romanian 1girl, as a curious killer ..."
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,7905bc95-2c3b-4c1e-8a3c-c0ab6295d2bf,6424802,R,"partial nudity, black hair, bracelet, dark ski...","cover art by angus mckie, scifi, futuristic, h..."


In [29]:
df.dropna(subset=["label"], inplace=True)

In [30]:
childDF = df[df["prompt"].str.contains("child", na=False)]
len(childDF)

2011

In [31]:
womanDF = df[df["tags"].str.contains("woman", na=False)]
manDF = df[df["tags"].str.contains("man", na=False)]
animalDF = df[df["tags"].str.contains("animal", na=False)]

# Function to sample 100 rows from each group
def sample_n_rows(group, n=1000):
    if len(group) < n:
        return group
    return group.sample(n=n, random_state=1)

# Group by 'original_level' and sample 100 rows from each group
sampled_womanDF = womanDF.groupby('label').apply(sample_n_rows).reset_index(drop=True)
sampled_manDF = manDF.groupby('label').apply(sample_n_rows).reset_index(drop=True)
sampled_animalDF = animalDF.groupby('label').apply(sample_n_rows).reset_index(drop=True)

data = pd.concat([sampled_womanDF, sampled_manDF, sampled_animalDF, childDF])
print(len(childDF), len(sampled_womanDF), len(sampled_manDF), len(sampled_animalDF), len(data))

2011 5000 5000 5000 17011


/var/folders/lj/vxfv4wp1351_qdc939xvvn0m0000gn/T/ipykernel_18665/680466905.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_womanDF = womanDF.groupby('label').apply(sample_n_rows).reset_index(drop=True)
/var/folders/lj/vxfv4wp1351_qdc939xvvn0m0000gn/T/ipykernel_18665/680466905.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_manDF = manDF.groupby('label').apply(sample_n_rows).reset_i

In [32]:
##uses the df to create a dictionary of the image urls and the labels
##Label indicates the subdirectory where the image will be saved
##for the csv 'final_eval_data' the label column is the same as 'reviewed_level'
label = list(data['label'])
urls = list(data['download_url'])

In [33]:
imageUrlDict = {}
for i in range(len(label)):
    if label[i] == "nan":
        continue
    key = label[i]
    value = urls[i]
    if key in imageUrlDict.keys():
        imageUrlDict[key].append(value)
    else:
        imageUrlDict[key] = [value]
            

In [34]:
##Confirm number of images to be downloaded per label
for k,v in imageUrlDict.items():
    print(k, len(v))


PG 4368
PG13 3116
R 3158
X 3264
XXX 3105


In [35]:
outputdir = os.path.join(dataDir, 'images_to_label')
# Call the function with your imageUrlDict and dataDir
download_images(imageUrlDict, outputdir)

Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/e36f660c-bb29-44a3-9057-2759d0532446/width=450/5371898.jpg to ./data/images_to_label/PG/5371898.jpg
Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/bb5ee037-b5fe-48e3-a978-97e6c8a4128e/width=450/7699028.jpg to ./data/images_to_label/PG/7699028.jpg
Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/0efe9d59-0076-45c1-bbe6-dc46e0422fc7/width=450/10924255.jpg to ./data/images_to_label/PG/10924255.jpg
Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/a201adf6-bb32-4097-9830-1a609398be7c/width=450/870874.jpg to ./data/images_to_label/PG/870874.jpg
Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/5a756dda-cd0f-42bd-8dac-fa7b2700806c/width=450/4362118.jpg to ./data/images_to_label/PG/4362118.jpg
Downloaded image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/af202d89-3ba6-4037-9928-ac1af34da93e/width=450/2219939.jpg to ./data/image

In [11]:
for i in os.listdir(outputdir):
    print(i, len(os.listdir(os.path.join(outputdir, i))))

PG 3268
PG13 6044
R 2406
X 3021
XXX 2750


## Check downloads

In [38]:
outputdir = os.path.join('./data/', 'images_to_label')
len(get_image_list(outputdir))

16869